In [1]:
import os
import torch

from bcnf.utils import get_dir, load_config
from bcnf.train import Trainer
from bcnf import CondRealNVP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
config = load_config(os.path.join(get_dir("configs"), "trainer_config.yaml"))

In [3]:
model = CondRealNVP.from_config(config).to(device)

/home/psaegert/miniconda3/envs/bcnf/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.244 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [4]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32


In [5]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: psaegert (balisticcnf). Use `wandb login --relogin` to force relogin


Using trajectory data for training. Shape: (500, 90, 3)

Created all nessesary objects



Train: 30.0743 - Val: 26.2870 (avg: 26.6092, min: 26.7273) | lr: 2.00e-04 - Patience: 2/500 - z: 0.0325 ± 0.8799:   1%|          | 327/50000 [00:50<2:06:20,  6.55it/s]       